# Regression Demo 

## Set up global variables and random_state

In [ ]:
import numpy as np
import torch
from matplotlib import pyplot as plt
from datetime import datetime
from opendataval.util import set_random_state

device = torch.device("cpu")
random_state = set_random_state(10)
date = datetime.now().strftime("%m-%d_%H:%M")

## Choose Experiment parameters

In [ ]:
from opendataval.dataloader.noisify import add_gauss_noise, mix_labels
regression_datasets = ["diabetes", "linnerud"]
regression_models = ['mlpregress', 'sklinreg']

dataset_name = regression_datasets[1]
train_count, valid_count, test_count = 100, 50, 50
add_noise_func = add_gauss_noise
noise_rate = 0.1
noise_kwargs = { 'noise_rate': noise_rate }


model_name = regression_models[1]
train_kwargs = {"epochs": 20, "batch_size": 50} if "sk" not in model_name else {}
metric_name = "mse"

## Get ExperimentMediator without specifying DataEvaluators

In [ ]:
from opendataval.experiment import ExperimentMediator

exper_med = ExperimentMediator.model_factory_setup(
    dataset_name=dataset_name,
    cache_dir="../data_files/",  # Since move inside demo directory
    force_download=False,
    train_count=train_count,
    valid_count=valid_count,
    test_count=test_count,
    add_noise_func=add_noise_func,
    noise_kwargs=noise_kwargs,
    random_state=random_state,
    model_name=model_name,
    device=device,
    train_kwargs=train_kwargs,
    metric_name=metric_name
)

## Data Evaluators

#### Lots of imports for the many Data Evaluators

In [ ]:
from opendataval.dataval.ame import AME
from opendataval.dataval.dvrl import DVRL
from opendataval.dataval.influence import InfluenceFunctionEval
from opendataval.dataval.knnshap import KNNShapley
from opendataval.dataval.oob import DataOob
from opendataval.dataval.margcontrib import LeaveOneOut
from opendataval.dataval.margcontrib import BetaShapley, DataShapley
from opendataval.dataval.margcontrib.banzhaf import DataBanzhaf

#### Set up a series of data evaluators

In [ ]:
data_evaluators = [
    AME(num_models=1500, random_state=random_state),
    DataOob(2000, random_state=random_state),  # 1000 samples
    DVRL(rl_epochs=4000, random_state=random_state, device=device),  # RL requires torch device
    InfluenceFunctionEval(5000, random_state=random_state),
    DataBanzhaf(5000, random_state=random_state),
    BetaShapley(gr_threshold=1.05, min_samples=500, cache_name="cached", random_state=random_state),
    DataShapley(gr_threshold=1.05, min_samples=500, cache_name="cached", random_state=random_state),
]

In [ ]:
exper_med = exper_med.compute_data_values(data_evaluators=data_evaluators)

## Running experiments on the data values

In [ ]:
from opendataval.experiment.exper_methods import (
    discover_corrupted_sample,
    noisy_detection,
    remove_high_low,
    increasing_bin_removal,
    save_dataval
)

# Saving the results
output_dir = f"../tmp/{dataset_name}_{noise_rate=}/{date}/"
exper_med.set_output_directory(output_dir)
output_dir

#### Discover corrupted sample

In [ ]:
fig = plt.figure(figsize=(15, 15))
df_resp, fig = exper_med.plot(discover_corrupted_sample, fig, col=2, save_output=True)

#### Noisy detection F1 scores

In [ ]:
exper_med.evaluate(noisy_detection, save_output=True)

### Removing high values and low values

In [ ]:
fig = plt.figure(figsize=(15, 15))
df_resp, fig = exper_med.plot(remove_high_low, fig, include_train=True, col=2, save_output=True)

### Increasing bin removal

In [ ]:
fig = plt.figure(figsize=(15, 15))
df_resp, fig = exper_med.plot(increasing_bin_removal, fig, include_train=True, col=2, save_output=True)

#### Save data values

In [ ]:
exper_med.evaluate(save_dataval, save_output=True)